# Fetching observations from Havvarsel Frost (havvarsel-frost.met.no) and Frost (frost.met.no) 

This notebook constructs a dataset containing oceanographic observations from havvarsel frost accomplished with neighboring atmospheric observations from frost

Havvarsel frost serves so far `temperature` measurements originated from badevann.no at seven beaches in the south of Norway. 
> Documentation:
> API documentation for obs/badevann http://havvarsel-frost.met.no/docs/apiref#/obs%2Fbadevann/obsBadevannGet 
> Datastructure described on http://havvarsel-frost.met.no/docs/dataset_badevann

From the frost server we retrieve observations from the `n` closest weather observation stations and include the values 
- `air_temperature`
TODO: 
- `wind_speed`
- `relative_humidity`
- `cloud_area_fraction` (which takes values from 0 = no clouds to 8 = fully clouded) 
- `sum(duration_of_sunshinePT1H)`
- `mean(surface_downwelling_shortwave_flux_in_air PT1H)`
- ...

> Documentation:
> API documentation for observations on https://frost.met.no/api.html#!/observations/observations 
> Available elements (params) are listed on https://frost.met.no/elementtable 
> Examples on Frost data manipulation with Python on https://frost.met.no/python_example.html
>
> See also:
> Complete documentation at https://frost.met.no/howto.html 
> Complete API reference at https://frost.met.no/api.html 


TODO:
 - Tune processing and storing of observational data sets (to suite whatever code that will use the data sets)
 - Prototype simple linear regression
 - Prototype simple ANN (with Tensorflow and Keras?)
 - ...
"""

In [ ]:
# Importing general libraries
import sys
import json
import datetime
import requests
from traceback import format_exc
import pandas as pd
import matplotlib.pyplot as plt
import numpy 

# Importing user defined functionalities
import FrostImporter


## Data from Havvarsel-Frost

The dataset construction relies on the selection of an observation site from the badevann dataset (`id` = 0....6, see badevann.no for further information) and the selection of a time range with `start_time` and `end_time`

In [ ]:
# Inputs
station_id     = "5"

start_time     = "2019-01-01T00:00"
end_time       = "2019-12-31T23:59"

FrostImporter = FrostImporter.FrostImporter(start_time=start_time, end_time=end_time)



In [ ]:
# Fetching data
location, data = FrostImporter.havvarsel_frost(station_id)


## Data from Frost

The frost server requires a client-id which everyone can create for himself on https://frost.met.no/auth/requestCredentials.html


In [ ]:
# Authentifiction id 
client_id = '3cf0c17c-9209-4504-910c-176366ad78ba' # for florian.beiser@sintef.no


### Identify relevant observation locations in Frost

As preparation for the actual timeseries extract, we have to identify the `station_id`s of those locations where measurements in frost are available


In [ ]:
# Retrieving closest station ids 
station_ids = FrostImporter.frost_location_ids(location, 5, client_id)

### Example

In the following, we show an example how data from frost can be added to the existing dataset

In [ ]:
# Load temperature
param = "air_temperature"

timeseries = FrostImporter.frost(station_ids[0],param)

# Select suitable times from the timeseries and add it to data
data4plot = FrostImporter.postprocess_frost(timeseries, "air_temp", data)
print(data4plot.head())

# save to csv
data4plot.to_csv("dataset.csv")

#plot
plt.plot(data4plot["water_temp"], label="water")
plt.plot(data4plot["air_temp0"], label="air")
plt.title("Temperature Timeseries")
plt.legend(loc="upper right")
plt.plot()


### Timeseries from Frost

Lastly, we download the timeseries for all selected stations and all selected parameter

WARNING: Not tested since it runs too long

In [ ]:
params = ["air_temperature", "wind_speed", "relative_humidity"]

# Adding measurement for all station_ids and all params to the dataset (as above)
for id in station_ids:
    for param in params:
        timeseries = FrostImporter.frost(id, param)
        data = FrostImporter.postprocess_frost(timeseries, id+param, data)




In [ ]:
data